In [1]:
import requests
import urllib
import re
import os
import json
import pandas
import xlwt

#Path to data file
os.chdir(r'E:\Work\ACW-Presentation')

#Date Range
year = range(2012,2021)  #2015-2020
month = ['01','02','03','04','05','06','07','08','09','10','11','12']  #MM

date_list = []
for each_year in year:
    for each_month in month:
        dates = str(each_year)+"-"+each_month
        date_list.append(dates)
        
end = "2020-12" #yyyy-MM 

In [2]:
def get_data(start_date,end_date,loc):

        url = "https://www.propertyvalue.co.nz/api/public/clapi/property/trends/house-values"
        headers = {
            'authority':'www.propertyvalue.co.nz',
            'method':'POST',
            'path':'/api/public/clapi/property/trends/house-values',
            'scheme':'https',
            'accept':'application/json, text/plain, */*',
            'accept-encoding':'gzip, deflate, br',
            'accept-language':'en-US,en;q=0.9,zh-CN;q=0.8,zh;q=0.7',
            'content-length':'130',
            'content-type':'application/json;charset=UTF-8',
            'origin':'https://www.propertyvalue.co.nz',
            'referer':'https://www.propertyvalue.co.nz/property-trends/residential-house-values',
            'sec-fetch-dest':'empty',
            'sec-fetch-mode':'cors',
            'sec-fetch-site':'same-origin',
            'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'
            }
        params = {
                'fromDate':start_date,
                'locations':loc,
                'toDate':end_date
               }      
        resp = requests.post(url,headers=headers,data=json.dumps(params))
        resp.encoding = "utf-8"
        result = str(resp.text)
        result_with_date = re.sub('"|\[|\]|\{|\}','',result) 
        final_result = result_with_date.split(':')
        
        value = final_result[2].split(',')[0]
        region = final_result[-2].replace(',percentChange','')
        percentage_change = final_result[-1]

        return{'Region':region,'Date':start_date, 'Value':value, '%':percentage_change}
        
def save_as_excel(info_list):
    workbook=xlwt.Workbook(encoding ='utf-8')
    worksheet = workbook.add_sheet('Data')
    titles = ['Region','Date','Value','Percentage_Change']
    keys = ['Region','Date','Value','%']
    for index, title in enumerate (titles):
        worksheet.write(0,index,title)
    for i, info in enumerate(info_list):
        for j, key in enumerate(keys):
            worksheet.write(i+1,j,info[key])
    workbook.save('NZ-PropertyPrice.xls')

def main(): 
    try:
        info_list = []
        NZ = [{'locationId':'260','locationType':'COUNTRY','suggestionName':'NEW ZEALAND'}]
        Auckland = [{'locationId':'7','locationType':'TA','suggestionName':'Auckland - City'}]
        Hamilton = [{'locationId':'16','locationType':'TA','suggestionName':'Hamilton City'}]
        Wellington =  [{'locationId':'47','locationType':'TA','suggestionName':'Wellington City'}]
        Christchurch =  [{'locationId':'60','locationType':'TA','suggestionName':'Christchurch City'}]
        
        for item in date_list:                
            info_list.append(get_data(item,end,NZ))
            info_list.append(get_data(item,end,Auckland))
            info_list.append(get_data(item,end,Hamilton))
            info_list.append(get_data(item,end,Wellington))
            info_list.append(get_data(item,end,Christchurch))
            save_as_excel(info_list)
    
    # the code that can cause the error
    except IndexError: # catch the error
        pass # pass will basically ignore it
         # and execution will continue on to whatever comes
         # after the try/except block

            
if __name__ == '__main__':
    main()